# Actividad 1 – Big Data  
## Creación de una Base de Datos Analítica para la empresa Clínica del Play S.A.S.  
### Estudiante: Justin Beckham Cardona Yepes  
### Materia: Big Data  
### Institución: IU Digital de Antioquia  